## Averaging attention distributions from the last four layers then classifying via a Linear Network

We extract the attention distribution directly from a model that has been pretrained on bias detection!

We then determine the predicted biased word, and use only the attention distribution associated with that extracted word for further classification. 

We average together the last four attention distributions of the input. The classification model is a simple super shallow neural network.

In [1]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import AttentionExperiment, ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params
from src.utils.attention_utils import reduce_attention_dist, return_idx_attention_dist
from src.utils.classification_utils import run_bootstrapping
from src.utils.shared_utils import get_bias_predictions

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
params = Params.read_params("linear-params.json")
print("model = {}".format(params.final_task['model']))
print("layers = {}".format(params.intermediary_task["attention"]["layers"]))
print("reducer = {}".format(params.intermediary_task["attention"]["reducer"]))

model = shallow_nn
layers = [8, 9, 10, 11]
reducer = concat


In [4]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dataset = ExperimentDataset.init_dataset(params.dataset)

03/21/2020 17:16:40 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
386it [00:00, 4838.66it/s]


### Attention Experiment: 
* Is a class that wraps useful methods to extract attention distributions from a given BERT-based model 
* The user has to provide in two config files: One to specify parameters for how the attention scores should be extracted and combined, and other to specify the intermediary model from which the attention scores should be extracted from
* The user needs to instantiate the attention experiment with a function that tells the model how to run 
 inference on the given model. The function header is specified below: 
 
 ``` def initialize_attention_experiment(cls, intermediary_task_params, dataset_params, verbose=False) ```
 


In [7]:
attention_dataloader = dataset.return_dataloader() 
attention_experiment = AttentionExperiment.initialize_attention_experiment(params.intermediary_task, params.dataset, verbose=True)

03/21/2020 17:17:43 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/21/2020 17:17:43 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/21/2020 17:17:43 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpz7xkjpqm
03/21/2020 17:17:49 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

Instantiated joint model with pretrained weights.
Succesfully loaded in attention experiment!


```extract_attention_scores()``` works out of the box because the attention experiment has the config file saved, and knows what BERT model to use/load in, which layers to extract the attention scores from, and what the inference function is that should be used on this particular BERT model.

Attention_scores is then a list of dictionaries. The keys in this dictionary are the specific layers of a BERT model and the values are the corresponding attention distributions extracted from that particular layer.

In [8]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

Getting the predictions from the BERT model trained to detect bias, and using those to index into the attention scores

In [9]:
bias_predictions = get_bias_predictions(dataset, params.intermediary_task, params.dataset, batch_size=2)

03/21/2020 17:18:36 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/21/2020 17:18:36 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/21/2020 17:18:36 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpinr8ytf_
03/21/2020 17:18:40 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

In [10]:
bias_indices = torch.argmax(bias_predictions == 1, dim=1).tolist()

In [11]:
attention_scores_indexed = return_idx_attention_dist(attention_scores, bias_indices)
reduced_attention = reduce_attention_dist(attention_scores_indexed, params.intermediary_task["attention"]["reducer"])
stacked_reduced_attention = torch.stack(reduced_attention).squeeze()
dataset.add_data(stacked_reduced_attention, "attention_dist")
dataset.shuffle_data()
print("reduced_attention[0].shape = {}".format(reduced_attention[0].shape))
print("stacked_reduced_attention.shape = {}".format(stacked_reduced_attention.shape))
dataset

reduced_attention[0].shape = torch.Size([320])
stacked_reduced_attention.shape = torch.Size([324, 320])


Length: 324 Keys: dict_keys(['pre_ids', 'masks', 'pre_lens', 'post_in_ids', 'post_out_ids', 'pre_tok_label_ids', 'post_tok_label_ids', 'rel_ids', 'pos_ids', 'categories', 'index', 'bias_label', 'attention_dist'])

### This is where the classification experiment starts

We create a classification experiment that contains useful methods for classifying bias based on the attention distributions. 

In [28]:
params = Params.read_params("linear-params.json")


In [29]:
classification_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [30]:
stats = run_bootstrapping(classification_experiment, dataset, params.final_task, num_bootstrap_iters=5, input_key="attention_dist", label_key="bias_label")

120 Epochs - 15 hidden

In [15]:
stats

{'auc': [(0.5657144171617856, 0.7520465686274509), 0.6299014809579826],
 'accuracy': [(0.58125, 0.6828124999999999), 0.628125]}

120 Epochs - 10 hidden

In [19]:
stats

{'auc': [(0.5272685244849862, 0.6495147377132671), 0.6025738274332044],
 'accuracy': [(0.5843750000000001, 0.671875), 0.64375]}

120 Epochs - 20 hidden

In [23]:
stats

{'auc': [(0.5991991341991343, 0.7007193786605551), 0.6294279661798844],
 'accuracy': [(0.6109375, 0.6828124999999999), 0.640625]}

120 Epochs - 30 hidden

In [27]:
stats

{'auc': [(0.4851242359291895, 0.6348720397249809), 0.5825010610304728],
 'accuracy': [(0.525, 0.6796875), 0.60625]}

200 Epochs - 15 hidden

In [31]:
stats

{'auc': [(0.5415788133435193, 0.6979890422077922), 0.6020815500933535],
 'accuracy': [(0.5796875, 0.740625), 0.646875]}

Changing Epoch Count 

In [32]:
classification_experiment.save_model_weights("linear-attention.weights")